In [2]:
import tensorflow as tf
import numpy as np
import tflearn.datasets.oxflower17 as oxflower17

curses is not supported on this machine (please install/reinstall curses for an optimal experience)
Instructions for updating:
Colocations handled automatically by placer.


In [3]:
X, Y = oxflower17.load_data(one_hot=True)

100.0% 60276736 / 60270631


('Succesfully downloaded', '17flowers.tgz', 60270631, 'bytes.')
File Extracted
Starting to parse images...
Parsing Done!


In [6]:
print('Shape of x: {}'.format(X.shape))
print('shape of y: {}'.format(Y.shape))
print("Sample y: {}".format(Y[0]))


categories = ['Buttercup', 'Colts Foot', 'Daffodil', 'Daisy', 'Dandelion', 'Fritillary', 'Iris', 'Pansy', 
              'Sunflower', 'WindFlower', 'SnowDrop', 'LilyValley', 'Bluebell', 'Crocus', 'Tigerlilly', 'Tulip',
              'CowSlip']

Shape of x: (1360, 224, 224, 3)
shape of y: (1360, 17)
Sample y: [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [8]:
paddings = ((0,0), (2,1), (2,1), (0,0))
X_train = np.pad(X, paddings, 'constant')
print("Sahape of X_train: {}".format(X_train.shape))

Sahape of X_train: (1360, 227, 227, 3)


In [15]:
from sklearn.model_selection import train_test_split

In [17]:
x_train, x_test, y_train, y_test = train_test_split(X_train, Y, test_size=0.1, random_state=1)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=1)
    
print("Train set size...\nx_train: {}\ny_train: {}\n".format(x_train.shape, y_train.shape))

print("Validation set size...\nx_val: {}\ny_val: {}\n".format(x_val.shape, y_val.shape))

print("Test set size...\nx_test: {}\ny_test: {}\n".format(x_test.shape, y_test.shape))

Train set size...
x_train: (1101, 227, 227, 3)
y_train: (1101, 17)

Validation set size...
x_val: (123, 227, 227, 3)
y_val: (123, 17)

Test set size...
x_test: (136, 227, 227, 3)
y_test: (136, 17)



In [31]:
## data is ready... 
def AlexNet(x):
    
    #.. input shape (227,227,3)
    """ filter shape (11,11), stride 4, no.of.filters = 96, activation is relu"""
    conv1_W = tf.Variable(tf.truncated_normal(mean = 0, stddev =1.0, shape = [11,11,3,96]))
    conv1_b = tf.Variable(tf.zeros(96))
    conv1 = tf.nn.conv2d(x, conv1_W, strides=[1,4,4,1], padding='VALID') + conv1_b
    conv1 = tf.nn.relu(conv1)
    
    """max pooling... filter size (3,3) and strides = 2"""
    pool1 = tf.nn.max_pool(conv1, ksize=[1,3,3,1], strides=[1,2,2,1], padding='VALID')
    ##....layer 1 is done....##
    
    #****
    #local response normalization
    #https://medium.com/@pechyonkin/key-deep-learning-architectures-alexnet-30bf607595f1
    #https://www.tensorflow.org/api_docs/python/tf/nn/local_response_normalization
    lrn1 = tf.nn.local_response_normalization(pool1, alpha=10**-4, beta=0.75, bias=2)
    #***
    
    #.. input size (27,27,96)
    """filter shape(5,5), padding = same, stride 1, filters= 256 """
    conv2_W = tf.Variable(tf.truncated_normal(mean = 0, stddev = 1.0, shape = [5,5,96,256]))
    conv2_b = tf.Variable(tf.zeros(256))
    conv2 = tf.nn.conv2d(lrn1, conv2_W, strides=[1,1,1,1], padding='SAME') + conv2_b
    conv2 = tf.nn.relu(conv2)
    
    """max pooling... filter_size (3,3) and stride = 2 """
    pool2 = tf.nn.max_pool(conv2, ksize=[1,3,3,1], strides=[1,2,2,1], padding='VALID')
    ##....layer 2 is done....##
    
    #***
    #Local Response Normalization
    lrn2 = tf.nn.local_response_normalization(pool2, alpha=10**-4, beta=0.75, bias=1)
    #***
    
    #.. input size (13,13,256)
    """filter shape(3,3), padding = same, stride = 1, filters = 384"""
    conv3_W = tf.Variable(tf.truncated_normal(mean = 0, stddev = 1.0, shape = [3,3,256,384]))
    conv3_b = tf.Variable(tf.zeros(384))
    conv3 = tf.nn.conv2d(lrn2, conv3_W, strides=[1,1,1,1], padding='SAME') + conv3_b
    conv3 = tf.nn.relu(conv3)
    ##....layer 3 is done....##
    
    #.. input size (13,13,384)
    """filter shape(3,3), padding = same, stride = 1, filters = 384"""
    conv4_W = tf.Variable(tf.truncated_normal(mean = 0, stddev = 1.0, shape = [3,3,384,384]))
    conv4_b = tf.Variable(tf.zeros(384))
    conv4 = tf.nn.conv2d(conv3, conv4_W, strides=[1,1,1,1], padding='SAME') + conv4_b
    conv4 = tf.nn.relu(conv4)
    ##....layer 4 is done....##
    
    #.. input size (13,13,384)
    """filter shape(3,3), padding = same, stride = 1, filters = 256"""
    conv5_W = tf.Variable(tf.truncated_normal(mean = 0, stddev = 1.0, shape = [3,3,384,256]))
    conv5_b = tf.Variable(tf.zeros(256))
    conv5 = tf.nn.conv2d(conv4, conv5_W, strides=[1,1,1,1], padding='SAME') + conv5_b
    conv5 = tf.nn.relu(conv5)
    
    #.. input size (13,13,256)
    """max pooling filter size = (3,3), stride = 2"""
    pool5 = tf.nn.max_pool(conv5, ksize=[1,3,3,1], strides=[1,2,2,1], padding='VALID')
    ##....layer 5 is done....##
    
    #.. input size (6,6,256) 
    """Fully connected layer with drop out(probability = 0.5)"""
    flat = tf.layers.flatten(pool5)
    
    #.. input size = 6*6*256=9216
    fc6_W = tf.Variable(tf.truncated_normal(mean = 0, stddev = 1.0, shape = (9216,4096)))
    fc6_b = tf.Variable(tf.zeros(4096))
    
    dropped1 = tf.nn.dropout(flat, keep_prob=0.5) #dropout
    fc6 = tf.matmul(dropped1, fc6_W) + fc6_b
    
    fc6 = tf.nn.relu(fc6)
    
    #.. input size 4096
    fc7_W = tf.Variable(tf.truncated_normal(mean = 0, stddev = 1.0, shape = (4096,4096)))
    fc7_b = tf.Variable(tf.zeros(4096))
    
    dropped2 = tf.nn.dropout(fc6, keep_prob=0.5) #dropout
    fc7 = tf.matmul(dropped2, fc7_W) + fc7_b
    
    fc7 = tf.nn.relu(fc7)
    
    #.. input size 4096
    fc8_W = tf.Variable(tf.truncated_normal(mean = 0, stddev = 1.0, shape = (4096, 1000)))
    fc8_b = tf.Variable(tf.zeros(1000))
    
    fc8 = tf.matmul(fc7, fc8_W) + fc8_b
    
    fc8 = tf.nn.relu(fc8)
    
    #.. input size 1000... output needs to be classified out of 17 classes
    fc9_W = tf.Variable(tf.truncated_normal(mean = 0, stddev = 1.0, shape = (1000,17)))
    fc9_b = tf.Variable(tf.zeros(17))
    
    fc9 = tf.matmul(fc8, fc9_W) + fc9_b 
    
    return fc9

In [32]:
x = tf.placeholder(tf.float32, (None, 227, 227, 3))
y = tf.placeholder(tf.int32, (None))


In [34]:
logits = AlexNet(x)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
training_operation = optimizer.minimize(loss_operation)

In [35]:
EPOCHS = 10
BATCH_SIZE = 128

correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

In [ ]:
import time

with tf.Session() as sess:
  
  sess.run(tf.global_variables_initializer())
  num_examples = len(x_train)
  
  print("Training....\n")
  print("For Batch size...", BATCH_SIZE)
  
  for i in range(EPOCHS):
    start = time.process_time()
    
    for offset in range(0, num_examples, BATCH_SIZE):
      end = offset + BATCH_SIZE

      batch_x, batch_y = x_train[offset:end], y_train[offset:end]
      sess.run(training_operation, feed_dict={x: batch_x, y: batch_y})
      
    

    validation_accuracy = evaluate(validation_x, validation_label)
    print("EPOCH {} ...".format(i+1))
    print("Validation Accuracy = {:.3f}".format(validation_accuracy))
    print("Time taken by Epoch: ",i ,time.process_time() - start)
    print()